<a href="https://colab.research.google.com/github/Spotinum/HUAChatBot/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DEBUGGING**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [ ]:
%xterm

# **MODEL DOWNLOAD**

In [ ]:
# Download all the libraries needed
!pip install -U httpx > /dev/null 2>&1
!pip install -U langchain_ollama > /dev/null 2>&1
!pip install langchain > /dev/null 2>&1
!pip install langchain_community > /dev/null 2>&1
# !pip install pypdf2 > /dev/null 2>&1
!pip install chromadb > /dev/null 2>&1
!pip install -U ollama > /dev/null 2>&1
%pip install "rerankers[transformers]" torch > /dev/null 2>&1
!pip installlangchain_huggingface > /dev/null 2>&1
!pip install rank_bm25 > /dev/null 2>&1

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import torch
from rerankers import Reranker

#Run ollama and direct the model file to the google drive
!sudo chown -R ollama:ollama /content/drive/MyDrive/models
!nohup bash -c 'OLLAMA_MODELS=/content/drive/MyDrive/models /usr/local/bin/ollama serve' &> ollama.log &



#For hugging face
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize the reranker
ranker = Reranker(
    model_name='mixedbread-ai/mxbai-rerank-xsmall-v1',
    model_type='cross-encoder',
    device=device
)


Loading TransformerRanker model mixedbread-ai/mxbai-rerank-xsmall-v1 (this message can be suppressed by setting verbose=0)
No dtype set
Using dtype torch.float32


config.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/142M [00:00<?, ?B/s]

Loaded model mixedbread-ai/mxbai-rerank-xsmall-v1
Using device cuda.
Using dtype torch.float32.


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

In [ ]:
#For debugging
!cat ollama.log

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIJ8VRY3u2gCzdd4napcnsznMQmCbN9BdyL9uREVh3DsZ

2024/12/05 17:56:16 routes.go:1197: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/content/drive/MyDrive/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* taur

# **PREPARATION**

In [ ]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader
from google.colab import userdata

parser = StrOutputParser()
MODEL = "mistral-nemo"
#MODEL = "aya"
model = OllamaLLM(model=MODEL)

JINA_API_KEY = userdata.get('JINA_API_KEY')




#Uses hugging face model for reranking (returns top 5)
def reRankingRetriever_local(query):
  retrieved_documents = retriever.invoke(query)
  documents_as_strings = [doc.page_content for doc in retrieved_documents]

  sorted_rows = ranker.rank(
    query=query,
    docs=documents_as_strings,
  )


  indices = [row.doc_id for row in sorted_rows]
  indices = indices[:5]


  filtered_documents = [retrieved_documents[i] for i in indices]
  return filtered_documents



#Uses Jina api for reranking (uses tokens)
def reRankingRetriever_API(query):
  retrieved_documents = retriever.invoke(query)
  documents_as_strings = [doc.page_content for doc in retrieved_documents]

  url = 'https://api.jina.ai/v1/rerank'
  headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {JINA_API_KEY}'
  }

  data = {
      "model": "jina-reranker-v2-base-multilingual",
      "query": query,
      "top_n": 5,
      "documents": documents_as_strings
  }

  # Make the API request
  response = requests.post(url, headers=headers, json=data)

  # Print the response
  response_data = response.json()

  # Extracting indices from results
  indices = [result['index'] for result in response_data['results']]

  filtered_documents = [retrieved_documents[i] for i in indices]
  return filtered_documents



**FOR LOADING THE VECTOR STORE FROM DB**

In [ ]:

persist_directory = '/content/drive/MyDrive/db'

embedding = OllamaEmbeddings(model="nomic-embed-text")

vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

retriever = vectordb.as_retriever(search_kwargs={"k": 35})
# retriever_plus = vectordb.as_retriever(search_kwargs={"k": 1})

**FOR CREATING A NEW VECTOR STORE DB FROM SCRAPED DATA**

In [ ]:
import os
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import hashlib
BASE_URL = "https://dit.hua.gr/index.php/en/home-dit"  # For English
EXCLUDE_URL_EN = "https://dit.hua.gr/index.php/en/infrastructure-2/"
EXCLUDE_URL_EL = "https://dit.hua.gr/index.php/el/infrastructure-2/"

file_path_exclude_url = '/content/drive/MyDrive/exclude_urls.txt'

#Read the file and load URLs into a list
with open(file_path_exclude_url, 'r') as file:
    EXCLUDE_URLS = [line.strip() for line in file if line.strip()]

visited_urls = set()  # To track visited URLs
visited_titles = set()  # To track visited titles
scrape_footers_headers = True  # Track header/footer scraping status
JINA_API_URL = "https://r.jina.ai"

def is_internal_link(url):
    """Check if the URL is internal and not in the exclusion list."""
    parsed_base = urlparse(BASE_URL)
    parsed_url = urlparse(url)
    return (parsed_url.netloc == parsed_base.netloc
            # and '/en/' in parsed_url.path
            and '/el/' in parsed_url.path
            and not url.startswith(EXCLUDE_URL_EL)
            and not url.startswith(EXCLUDE_URL_EN)
            and url not in EXCLUDE_URLS)

def scrape_page(url):
    """Scrape a single page and extract its content."""
    global scrape_footers_headers

    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')


        title = soup.title.string if soup.title else "No Title"

        if title in visited_titles:
            print(f"Skipping duplicate title: {title}")
            return


        headers = {
            "Authorization": f"Bearer {JINA_API_KEY}",
            "X-Retain-Images": "none",
            "X-Remove-Selector": "header, .class, #id, footer, .cky-consent-bar, .cky-preference-center, #sp-header, #sp-footer, .offcanvas-menu",
            "X-Return-Format": "markdown"
        }

        response.raise_for_status()
        responseJina = requests.get(f"{JINA_API_URL}/{url}", headers=headers)

        # Check if the response is successful
        if responseJina.status_code == 200:
            print(f"Scraping: {url}")
            # Parse the plain text response
            raw_text = responseJina.text


            if title:
                safe_title = title.replace('/', '_').replace(' ', '_')
            else:
                main_heading = soup.find('h1')
                if main_heading:
                    safe_title = main_heading.text.strip().replace('/', '_').replace(' ', '_')
                else:
                    content_hash = hashlib.md5(response.text.encode('utf-8')).hexdigest()
                    safe_title = f"page_{content_hash}"

            filepath = f"{safe_title}.md"

            # Save the Markdown content to a file
            with open(f"/content/drive/MyDrive/scraped_pages_allDit/{filepath}", "w", encoding="utf-8") as file:
                file.write(raw_text)

        else:
            print(f"Failed to fetch the content: {response.status_code} - {response.reason}")


        visited_titles.add(title)

        # Extract all the links in the page to scrape recursively
        links = [a['href'] for a in soup.find_all('a', href=True)]
        for link in links:
            full_url = urljoin(BASE_URL, link)  # Convert relative URL to absolute
            if is_internal_link(full_url) and full_url not in visited_urls:
                visited_urls.add(full_url)
                scrape_page(full_url)  # Recursively scrape linked pages

    except Exception as e:
        print(f"Error scraping {url}: {e}")

def main():
    # Create a directory to save the scraped files
    if not os.path.exists("/content/drive/MyDrive/scraped_pages_allDit"):
        os.makedirs("/content/drive/MyDrive/scraped_pages_allDit")

    # Start scraping from the base URL
    visited_urls.add(BASE_URL)
    scrape_page(BASE_URL)

    print("Scraping complete.")

# Run the main function
main()

Scraping: https://dit.hua.gr/index.php/en/home-dit
Scraping: https://dit.hua.gr/index.php/el/home-dit-gr
Skipping duplicate title: Το Τμήμα
Scraping: https://dit.hua.gr/index.php/el/department-gr/faculty-members
Scraping: https://dit.hua.gr/index.php/el/department-gr/advisory-board-gr
Scraping: https://dit.hua.gr/index.php/el/department-gr/secretariat
Scraping: https://dit.hua.gr/index.php/el/department-gr/student-services
Scraping: https://dit.hua.gr/index.php/el/department-gr/e-platforms-gr
Scraping: https://dit.hua.gr/index.php/el/department-gr/facilities
Scraping: https://dit.hua.gr/index.php/el/department-gr/governance
Scraping: https://dit.hua.gr/index.php/el/department-gr/contact-access
Scraping: https://dit.hua.gr/index.php/el/department-gr/erasmus
Scraping: https://dit.hua.gr/index.php/el/studies/undergraduate-studies
Scraping: https://dit.hua.gr/index.php/el/studies/doctorate-studies
Scraping: https://dit.hua.gr/index.php/el/studies/metadidaktoriki-ereuna
Scraping: https://di

In [ ]:
!pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import TextSplitter


class MarkdownTitleTextSplitter(TextSplitter):
    def split_text(self, text: str):
        # Split the document into sections based on Markdown headings (e.g., `###`)
        sections = []
        current_section = []
        for line in text.splitlines():
            if line.startswith("###"):  # Treat `###` as a new section
                if current_section:  # Save the current section if it exists
                    sections.append("\n".join(current_section))
                current_section = [line]  # Start a new section
            else:
                current_section.append(line)

        # Add the final section if there's remaining content
        if current_section:
            sections.append("\n".join(current_section))

        return sections


directory_path = '/content/drive/MyDrive/scraped_pages_en'

# Load the documents
loader = DirectoryLoader(directory_path, glob="*.md", loader_cls=TextLoader)
documents = loader.load()

# Apply the custom Markdown splitter
markdown_splitter = MarkdownTitleTextSplitter(chunk_size=1500, chunk_overlap=100)
texts = markdown_splitter.split_documents(documents)

# Persist embeddings with Chroma
persist_directory = 'db'

embedding = OllamaEmbeddings(model="paraphrase-multilingual")

# model_name = "mixedbread-ai/mxbai-embed-large-v1"
# embedding = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda', 'trust_remote_code': True},
#     encode_kwargs={'normalize_embeddings': False}
# )


vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

retriever = vectordb.as_retriever(search_kwargs={"k": 35})

In [ ]:
retriever.invoke("what are the subjects of the 8th semetser?")

In [ ]:

from langchain.retrievers import BM25Retriever, EnsembleRetriever


bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2


ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

# **GENERATION**

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
        You are an assistant for question-answering tasks called HUA-Bot.
        Hua stands for Harokopio University of Athens
        Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
        If you are not sure about something in the context and can't find another answer just give the answer you aren't sure about.
        Use three sentences maximum and keep the answer concise
        Question: {question}
        Context: {context}
        Answer: """,
        input_variables = ["question","document"],
    )


# prompt = PromptTemplate(
#     template="""
#         <|begin_of_text|><|start_header_id|>system<|end_header_id|> Είστε ένας βοηθός για ερωτήσεις και απαντήσεις με την ονομασία HUA-Bot.
#         Το HUA σημαίνει Χαροκόπειο Πανεπιστήμιο Αθηνών.
#         Χρησιμοποιήστε τα παρακάτω αποσπάσματα από το πλαίσιο που παρέχεται για να απαντήσετε στην ερώτηση. Αν δεν γνωρίζετε την απάντηση, απλώς πείτε ότι δεν την γνωρίζετε.
#         Αν δεν είστε σίγουρος για κάτι στο πλαίσιο και δεν μπορείτε να βρείτε άλλη απάντηση, απλώς δώστε την απάντηση που δεν είστε σίγουρος.
#         Χρησιμοποιήστε μέγιστο τριών προτάσεων και κρατήστε την απάντηση σύντομη. <|eot_id|><|start_header_id|>user<|end_header_id|>
#         Ερώτηση: {question}
#         Πλαίσιο: {context}
#         Απάντηση: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#         input_variables = ["question", "document"],
#     )



chain = prompt | model | parser
# # question = "Πότε ιδρυθηκε το τμήμα;"
# docs = retriever.invoke(translated_question)
# print(docs)
# print("\n\n\n")
# generation = chain.invoke({"context":docs, "question":translated_question})
# print(generation)


# **TEST**

In [ ]:
def get_answer_with_validation(question):
    docs = reRankingRetriever_local(question)
    generation = chain.invoke({"context": docs, "question": question})
    print(generation)

In [ ]:


questions = [
    "What are the roles of Malvina Vamvakari in the Department?",
    "What is the link to the study guide?",
    "Can you give me the link to the courses schedule?",
    "when was the department founded",
    "who is the dean of the Department",
    "What are the subjects of the first Semester",
    "Tell me about the erasmus project",
    "Does the Department have an e-class?",
    "Tell me about the subject Discrete Math",
    "Tell me Christos Diou email",
    "who is the Head of the Department's Secretariat",
    "Where is the Department Located?",
    "What are the graduation requirements for the undergraduate program?"
]

# questions = [
#     "Πότε ιδρύθηκε το Τμήμα;",
#     "Ποιος είναι ο Κοσμήτορας του Τμήματος;",
#     "Ποια είναι τα μαθήματα του πρώτου εξαμήνου;",
#     "Πείτε μου για το πρόγραμμα Erasmus.",
#     "Έχει το Τμήμα ηλεκτρονική τάξη (e-class);",
#     "Πείτε μου για το μάθημα Διακριτά Μαθηματικά.",
#     "Ποιο είναι το email του Χρήστου Διού;",
#     "Ποιος είναι ο Προϊστάμενος της Γραμματείας του Τμήματος;",
#     "Πού βρίσκεται το Τμήμα;",
#     "Ποιες είναι οι απαιτήσεις αποφοίτησης για το προπτυχιακό πρόγραμμα;",
#     "Παρέχονται στους φοιτητές υπολογιστικοί πόροι ή άδειες χρήσης λογισμικού;",
#     "Συμμετέχει το Τμήμα σε προγράμματα ανταλλαγής Erasmus+;",
#     "Υπάρχουν ευκαιρίες εθελοντισμού εντός του Τμήματος;"
# ]

for idx, question in enumerate(questions, start=1):
    print(f"Question{idx}: {question}")
    get_answer_with_validation(question)


Question1: What are the roles of Malvina Vamvakari in the Department?
Based on the provided documents, here are some key points about the Secretariat of the Department of Informatics and Telematics at Harokopio University:

1. **Foteini - Maria Mine**:
   - Works at the Secretariat since 2022.
   - Has a Bachelor's degree in International and European Studies and a Master's degree in Maritime Studies from the University of Piraeus.

2. **Nikolaos Sfakianos**:
   - Works at the Secretariat since 2022, supporting activities related to quality management and evaluation of Postgraduate Programs.
   - Has a Bachelor's degree in Sociology and two Master's degrees in Sociology and Social Psychology from Panteion University.
   - Is currently working on his PhD thesis focused on social comparison in general use social networking media.

3. **Maria Sdraka** (PhD candidate):
   - Focuses on developing and applying Machine Learning techniques on big Earth Observation data for natural disaster ass

In [ ]:

while True:
    question = input("Enter your question: ")
    if question.lower() == "exit":
        break
    get_answer_with_validation(question)

Enter your question: give me the eclass link of the subject machine learning
I don't know the exact eClass link for Machine Learning as it was not provided in the given context. However, you can usually find course links at http://eclass.hua.gr/courses/, using the course code if available.
Enter your question: give me the eclass link of the subject machine learning of the 7th semester
The eClass link for Machine Learning (ΕΠ34) in the 7th semester is not provided directly in the given context. However, you can find it on the list of undergraduate studies courses: https://dit.hua.gr/index.php/en/studies/undergraduate-studies?view=article&id=1918:ep34-machine-learning-and-applications&catid=95
Enter your question: exit


# **JUNK**

In [ ]:

MODEL = "mistral-nemo"
# MODEL = "aya"
model_check = OllamaLLM(model=MODEL, max_tokens=1)

prompt_check = PromptTemplate(
    template="""
You are a grader assessing whether
an answer is grounded in / supported by a set of facts based of the question given. Give a binary score 'yes' or 'no' score to indicate
whether the answer is grounded in / supported by a set of facts based on the question given. Only give a 'yes' or 'no' answer
Here are the facts:
\n ======== \n
{documents}
\n ======== \n
Here is the answer: {generation}
\n ======== \n
Here is the question: {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents", "question"],
)

# prompt_check = PromptTemplate(
#     template="""
# <|begin_of_text|><|start_header_id|>system<|end_header_id|> Είστε ένας αξιολογητής που εξετάζει αν
# μία απάντηση βασίζεται ή υποστηρίζεται από ένα σύνολο γεγονότων με βάση την ερώτηση που δόθηκε. Δώστε μια δυαδική απάντηση 'yes' ή 'no' για να υποδείξετε
# αν η απάντηση βασίζεται ή υποστηρίζεται από ένα σύνολο γεγονότων με βάση την ερώτηση που δόθηκε. Δώστε μόνο μια απάντηση 'yes' ή 'no'. <|eot_id|><|start_header_id|>user<|end_header_id|>
# Εδώ είναι τα γεγονότα:
# \n ======== \n
# {documents}
# \n ======== \n
# Εδώ είναι η απάντηση: {generation}
# \n ======== \n
# Εδώ είναι η ερώτηση: {question}
# <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["generation", "documents", "question"],
# )


hallucination_grader = prompt_check | model_check | parser

# hallucination_grader.invoke({"documents": docs, "generation": generation, "question": translated_question})


In [ ]:
import os
BASE_URL = "https://dit.hua.gr/index.php/en/home-dit"  # For English
EXCLUDE_URL = "https://dit.hua.gr/index.php/en/infrastructure-2/"
EXCLUDE_URLS = [
    "https://dit.hua.gr/index.php/en/research/scientific-lectures?view=article&id=10",
    "https://dit.hua.gr/index.php/en/studies/undergraduate-studies?view=article&id=382:2010-10-13-07-52-08&catid=45:diploma-theses",
    "https://dit.hua.gr/index.php/en/studies/undergraduate-studies?view=article&id=287:2010-10-13-07-52-08&catid=45:diploma-theses",
    "https://dit.hua.gr/index.php/en/studies/undergraduate-studies?view=article&id=223:2010-10-13-07-52-08&catid=45:diploma-theses",
    "https://dit.hua.gr/index.php/en/studies/undergraduate-studies?view=article&id=186:2010-10-13-07-52-08&catid=45:diploma-theses",
    "https://dit.hua.gr/index.php/en/studies/undergraduate-studies?amp;view=article&amp;id=839",
    "https://dit.hua.gr/index.php/en/studies/undergraduate-studies?amp;view=article&amp;id=769"
]

visited_urls = set()  # To track visited URLs
visited_titles = set()  # To track visited titles
scrape_footers_headers = True  # Track header/footer scraping status

def is_internal_link(url):
    """Check if the URL is internal and not in the exclusion list."""
    parsed_base = urlparse(BASE_URL)
    parsed_url = urlparse(url)
    return (parsed_url.netloc == parsed_base.netloc
            and '/en/' in parsed_url.path
            and not url.startswith(EXCLUDE_URL)
            and url not in EXCLUDE_URLS)

def scrape_page(url):
    """Scrape a single page and extract its content."""
    global scrape_footers_headers

    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        if not scrape_footers_headers:
            for footer in soup.find_all('footer'):
                footer.decompose()  # Remove the footer from the HTML
            for header in soup.find_all('header'):
                header.decompose()  # Remove the header from the HTML
        else:
            scrape_footers_headers = False

        title = soup.title.string if soup.title else "No Title"

        if title in visited_titles:
            print(f"Skipping duplicate title: {title}")
            return
        print(f"Scraping: {url}")

        for a_tag in soup.find_all('a', href=True):
            link = a_tag['href']
            # Replace the text inside the <a> tag to include the link next to the text
            a_tag.string = f"{a_tag.string} ({link})" if a_tag.string else f"({link})"

        # Clean the title to create a safe filename
        safe_title = title.replace('/', '_').replace(' ', '_')

        # Write content to a separate file for each page
        content = soup.get_text(separator="/n", strip=True)

        # Save the content to a text file
        with open(f"scraped_pages/{safe_title}.txt", "w", encoding="utf-8") as file:
            # file.write(f"URL: {url}\n")
            # file.write(f"Title: {title}\n")
            file.write(f"Content:\n{content}\n")

        visited_titles.add(title)

        # Extract all the links in the page to scrape recursively
        links = [a['href'] for a in soup.find_all('a', href=True)]
        for link in links:
            full_url = urljoin(BASE_URL, link)  # Convert relative URL to absolute
            if is_internal_link(full_url) and full_url not in visited_urls:
                visited_urls.add(full_url)
                scrape_page(full_url)  # Recursively scrape linked pages

    except Exception as e:
        print(f"Error scraping {url}: {e}")

def main():
    # Create a directory to save the scraped files
    if not os.path.exists("scraped_pages"):
        os.makedirs("scraped_pages")

    # Start scraping from the base URL
    visited_urls.add(BASE_URL)
    scrape_page(BASE_URL)

    print("Scraping complete.")

# Run the main function
main()

# **EVALUATIONS**

In [ ]:
#RERANKER EVAL
import time
%%time
query = "What are the subjects of the 8th semester?"
#query = "what is christos diou email?"
#query = "what is the undergraduate secretery phone?"
#query = "what is the office of Papadopoulos and what is his phone number?"
#query = "what is the e-class link for the subject Machine Learning and Applications"
retrieved_documents = retriever.invoke(query)
documents_as_strings = [doc.page_content for doc in retrieved_documents]

sorted_rows = ranker_mxbai_rerank_large_v1.rank(
  query=query,
  docs=documents_as_strings,
)


indices = [row.doc_id for row in sorted_rows]
indices = indices[:5]


filtered_documents = [retrieved_documents[i] for i in indices]

print(filtered_documents)
generation = chain.invoke({"context":filtered_documents, "question":query})
print(generation)

UsageError: Line magic function `%%time` not found.
